In [1]:
import os
import sys
import pybedtools
import re
import subprocess
import argparse
from statistics import mode, multimode
import find_feature
import find_tu
from collections import Counter
import pandas as pd
# import annotate_tss as at

In [ ]:

filter_tss="/scratch/rx32940/minION/polyA_directRNA/TU_Annotation/direct_output/tss/chromIlag.filtered20.TSS.tab"
gff="/scratch/rx32940/minION/polyA_cDNA/map/genome/reference/GCF_000007685.1_ASM768v1_genomic.gff"
output="/scratch/rx32940/minION/polyA_directRNA/TU_Annotation/direct_output/tss/chromIlead.annotate20.TSS.tab"

all_genes=at.get_genes(gff) 
all_tss=at.get_tss(all_genes, filter_tss)
tss_anot=at.annotate_tss(all_genes, all_tss)
anot_df = at.annotate_gTSS(tss_anot)[["chrom","start","end","tss_type","cov", "strand","gTSS_anot" ,"tssGene", "numSamples", "geneFrom", "max_cov"]]




In [3]:
bam="/scratch/rx32940/minION/polyA_directRNA/map/genome/bam/Q29_Copenhageni_Basecalled_May_22_2020_Direct-RNA_rna_filtered.linear.bam"
gff="/scratch/rx32940/minION/polyA_cDNA/map/genome/reference/GCF_000007685.1_ASM768v1_genomic.gff"
output="/scratch/rx32940/minION/polyA_directRNA/TU_Annotation/input/"



In [4]:
tss_obj=find_feature.FindFeature(bam=bam, gff=gff, output=output, feature="TSS", unawarecovfilter = 0.9)
tss_dict = tss_obj.find_features()
# tts_obj=find_feature.FindFeature(bam=bam, gff=gff, output=output, feature="TTS")
# tts_dict = tts_obj.find_features()



start finding TSS with gene aware approach:
start finding TSS with gene unaware approach.
combining TSS from gene aware and unaware approach.
finding TSS in the current alignment:
new bed file generated from bam:True
threhold:10
Gene aware coverage filter:0.300000

number of TSS identified from each chrom and strand:
 {'NC_005823.1': {'+': 3503, '-': 4009}, 'NC_005824.1': {'+': 296, '-': 347}} 


In [ ]:
test = tss_obj.pos_cov
chrom="NC_005824.1"
strand="-"
test[chrom][strand][350181]

tss_obj.all_reads[chrom][strand][0]

In [ ]:
tu_obj = find_tu.FindTU(bam=bam, gff=gff)
tu_obj.find_operon()

In [ ]:
all_orfs={}
for chrom in tts_dict.keys():
    all_orfs[chrom]={}
    for strand in tts_dict[chrom].keys():
        all_orfs[chrom][strand]=[]
        # print(strand)
        if strand == "+":
            for tss in tss_dict[chrom][strand][0:5]:
                # print(tss)
                tss_start=tss[0] 
                tss_end=tss[1]
                tss_gene=tss[2]
                i=0
                orf_list=[]
                tts_start= tts_dict[chrom][strand][i][0]
                tts_end= tts_dict[chrom][strand][i][1]
                tts_gene=tts_dict[chrom][strand][i][2]
                enlonged =0
                tss_found=0
                print("current tss: " + str((tss_start , tss_end)))
                while i < (len(tts_dict[chrom][strand])-1) and tts_end <= tss_end: # loop while the tss cluster's start (5') is smaller than the start of tts cluster (3') 
                    print("current tts" + str((tts_start, tts_end)))
                    if tts_start >= tss_start and tts_end <= tss_end: # if the cluster of tss and tts overlap
                        # the current tu is enlonged with the tts cluster
                        # use the enlonged version of tu to check overlapping with the next tts cluster
                        enlonged += 1 
                        tss_found +=1
                        tss_start = tss_start 
                        tss_end = tts_start if tts_start > tss_end else tss_end
                        tss_gene = tss_gene + ":" + tts_gene if tss_gene != tts_gene else tss_gene
                        print("enlonged tu: " + str((tss_start, tss_end)))
                    elif enlonged > 0:
                        orf_list.append((tss_start, tss_end, tss_gene))
                        print("current orf:" + str((tss_start, tss_end)))
                        print(enlonged)
                        enlonged = 0
                    i +=1
                    tts_start= tts_dict[chrom][strand][i][0]
                    tts_end= tts_dict[chrom][strand][i][1]
                    tts_gene=tts_dict[chrom][strand][i][2]
                orf_list.append((tss_start, tss_end, tss_gene))
                all_orfs[chrom][strand] = all_orfs[chrom][strand] + orf_list
                

In [ ]:
all_reads = tss_obj.all_reads

all_genes = tss_obj.all_genes

all_operons={}
for chrom in all_reads.keys():
    all_operons[chrom]={}
    for strand in all_reads[chrom].keys():
        all_operons[chrom][strand] = []
        operon_list=[]
        if strand == "-": # lag strand reads has second position as 5'
            read_start=1
            read_end=0
        else:
            read_start=0
            read_end=1
        for read in all_reads[chrom][strand]: # loop through each read
            cur_operon_start = read[read_start]
            cur_operon_end = read[read_end]
            cur_operon_genes = ""
            gene_in_operon = 0
            i=0
            while i < len(all_genes[chrom][strand]): # rotating through genes (read is stationary)
                cur_gene = all_genes[chrom][strand][i]
                if cur_gene[1] < cur_operon_start: # genes end is before read start
                    i += 1
                elif cur_gene[0]> cur_operon_end : # if gene start after read ends, then no more future gene will overlap with this read if gene list sorted
                    cur_operon = (cur_operon_start, cur_operon_end, gene_in_operon, cur_operon_genes[1:])
                    i = len(all_genes[chrom][strand]) # stop looping through genes
                else: # gene over lap with current read
                    gene_in_operon +=1 
                    cur_operon_genes = cur_operon_genes + ":"+ cur_gene[2] 
                    i += 1
            all_operons[chrom][strand].append(cur_operon)
        

In [ ]:
# all_operons

with open(os.path.join(output, "Q29_Copenhageni_Basecalled_May_22_2020_Direct-RNA_rna_filtered.linear.operons.tab"), "w") as ow:
    ow.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\n"%("chrom" ,"strand","start", "end","numGenes","numOfReads", "genesIncluded")) 
    operon_cov_dict={}
    for chrom in all_operons.keys():
        operon_cov_dict[chrom]={}
        for strand in all_operons[chrom].keys():
            operon_cov_dict[chrom][strand]=[]
            # get unique operons (with different genes included) and get number of reads supporting each operon
            genes_in_operons=[x[3] for x in all_operons[chrom][strand]]
            operons_cov = Counter(genes_in_operons)
            for op in operons_cov.keys():
                op_start= min([x[0] for x in all_operons[chrom][strand] if x[3] == op])
                op_end = max([x[1] for x in all_operons[chrom][strand] if x[3] == op])
                op_spread=[x[2] for x in all_operons[chrom][strand] if x[3] == op][0]
                op_cov = operons_cov[op]
                operon_cov_dict[chrom][strand].append((op_start, op_end,op_spread ,op_cov, op))
                ow.write("%s\t%s\t%d\t%d\t%d\t%d\t%s\n"%(chrom, strand, op_start, op_end,op_spread ,op_cov, op)) 
                
            

In [ ]:
operon_cov_dict

# all_operons

In [ ]:


genes_in_operons = [x[3] for x in operon_list]

operons_cov = Counter(genes_in_operons)
with open(os.path.join(output, "Q29_Copenhageni_Basecalled_May_22_2020_Direct-RNA_rna_filtered.linear.operons.tav"), "w") as ow:
    ow.write("%s\t%s\t%s\t%s"%("chrom" ,"strand""Operons", "numOfReads"))
    for chrom in operon_cov_dict.keys():
        for strand in operon_cov_dict[chrom].keys():
            
    

In [ ]:
operon_list
cur_operon_genes
len(all_reads[chrom][strand])
test = list(set(all_reads[chrom][strand]))
test.sort()
test[3258:5200]
chrom="NC_005823.1"
strand="-"
distinct_read = list(set(all_reads[chrom][strand]))
distinct_read.sort()
all_genes[chrom]["-"]
